In [ ]:
!pip install transformers
!pip install diffusers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.0/852.0 kB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel, pipeline
from PIL import Image
import requests
import random



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
import torch

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
unmasker = pipeline('fill-mask', model='bert-base-uncased')
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")



Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def diff(prompt):
  image = pipe(prompt).images[0]

  image.save("image.png")

  words = prompt.split()
  index = random.randint(0, len(words)-1)
  punct = ['.', '!', ',']
  blocked_words = ['photo', 'picture', 'image']

  unmasked_text = prompt


  while unmasked_text == prompt:
    while (len(words[index]) < 3 and words[index] not in blocked_words):
      index = random.randint(0, len(words)-1)

    words[index] = "[MASK]"
    new_text = ' '.join(words)
    if new_text[-1] not in punct:
      new_text += '.'

    if prompt[-1] not in punct:
      prompt += '.'


    unmasked = unmasker(new_text)

    unmasked_text = unmasked[0]['sequence']


  
  inputs = processor(text=[prompt, unmasked_text], images=image, return_tensors="pt", padding=True)

  outputs = clip_model(**inputs)
  logits_per_image = outputs.logits_per_image # this is the image-text similarity score
  probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

  l = ["original prompt:", prompt, "\n masked prompt:", new_text, "\n unmasked prompt:", unmasked_text, "\n Probability of original prompt: ", str(round(100*(torch.Tensor.item(probs[0][0])),2)), "% \nProbability of modified prompt: ", str(round(100*(torch.Tensor.item(probs[0][1])),2)), "% \n"]

  text_output = ''.join(l)

  if probs[0][0] > probs[0][1]:
    text_output += "predicted original prompt"
  elif probs[0][0] == probs[0][1]:
    text_output+= "Both prompts have equal probabilities, this may mean that the prompts are the same."
  else:
    text_output+= "predicted modified prompt"
  return image, text_output





In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=diff,
    inputs=gr.Textbox(lines=1, placeholder="Prompt..."),
    outputs=[gr.Image(shape=(512,512)), "text"],

)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>